In [31]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [32]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [33]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [34]:

dependent=dataset['classification_yes']


In [35]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [36]:
from sklearn.preprocessing import StandardScaler
SC=StandardScaler()
x_train=SC.fit_transform(x_train)
x_test=SC.transform(x_test)

In [42]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid= {'kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'], 'gamma':['scale', 'auto'], 'class_weight':['dict','balanced'],'C':[0.001,0.01,0.1,1.0,10.0,100.0,1000.0]}

grid=GridSearchCV(SVC(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

ValueError: X should be a square kernel matrix

In [8]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [9]:
re=grid.cv_results_
print(re)


NameError: name 'grid' is not defined

In [ ]:
grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


In [ ]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

In [ ]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


In [ ]:
table=pd.DataFrame.from_dict(re)

In [ ]:
table